## 03 - Modelling

## Optional - Colab Setup

In [ ]:
def upgrade_runtime_ram():
    meminfo = subprocess.getoutput('cat /proc/meminfo').split('\n')

    memory_info = {entry.split(':')[0]: int(entry.split(':')[1].replace(' kB','').strip()) for entry in meminfo}

    if memory_info['MemTotal'] > 17000000:
        return

    a = []
    while(1):
        a.append('1')

In [ ]:
def restart_runtime():
    os.kill(os.getpid(), 9)

In [ ]:
def setup_rapids():
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    device_name = pynvml.nvmlDeviceGetName(handle)
    if (device_name != b'Tesla T4') and (device_name != b'Tesla P4') and (device_name != b'Tesla P100-PCIE-16GB'):
        print("Wrong GPU - Restarting Runtime")
        restart_runtime()


    # clone RAPIDS AI rapidsai-csp-utils scripts repo
    !git clone https://github.com/rapidsai/rapidsai-csp-utils.git

    # install RAPIDS
    !bash rapidsai-csp-utils/colab/rapids-colab.sh 0.13


    # set necessary environment variables 
    dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
    sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
    sys.path

    # update pyarrow & modules 
    exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

In [ ]:
def setup_conda():
    if not 'Miniconda3-4.5.4-Linux-x86_64.sh' in os.listdir():
        !wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh && bash Miniconda3-4.5.4-Linux-x86_64.sh -bfp /usr/local

    if not ('EPFL-Capstone-Project' in os.listdir()) and (os.getcwd().split('/')[-1] != 'EPFL-Capstone-Project'):
        !git clone https://github.com/helmigsimon/EPFL-Capstone-Project  
    if 'EPFL-Capstone-Project' in os.listdir():
        os.chdir('EPFL-Capstone-Project')

    !conda env create -f environment.yml
    !conda activate exts-ml

In [ ]:
def setup_drive():
    #Mounting Google Drive
    global drive
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
try:
    import sys,os,subprocess
    
    upgrade_runtime_ram()
    setup_drive()

    #Setting up PyPi Packages
    !pip install geopandas sparse-dot-topn pdpipe category-encoders
    import geopandas as gpd
    import sparse_dot_topn.sparse_dot_topn as ct
    import pdpipe as pdp
    import category_encoders

    #Setting up Conda Packages
    setup_conda()
    
    #Initializing NLTK
    import nltk
    nltk.download('stopwords')
    nltk.download('punkt')
    
    #Setting up RAPIDS AI
    import pynvml
    setup_rapids()
    
    from cuml import UMAP
    
except ModuleNotFoundError as e:
    print(e)
    print('Not in colab environment, continuing to run locally')
    from umap import UMAP

## Imports

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split,  StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
tqdm.pandas()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer

In [ ]:
from lib.transformers import *
from lib.pipelines import *
from lib.processing import save_to_pkl, load_from_pkl
from data.util.paths import DATA_PATH

In [ ]:
from category_encoders.leave_one_out import LeaveOneOutEncoder

## Setting up DataFrame

In [ ]:
api_df = load_from_pkl('api',DATA_PATH)
extracted_df = load_from_pkl('extracted',DATA_PATH)

In [ ]:
api_df = api_pipe.fit_transform(api_df)
extracted_df = extracted_pipe.fit_transform(extracted_df)

In [ ]:
#Remove outliers from extracted_df
extracted_df = OutlierRemover('market_value').fit_transform(extracted_df)

## Reducing Dimensionality of Image Embeddings

In [ ]:
with np.load(os.path.join(DATA_PATH,'high_level_features_labelled.npz')) as data:
    image_embedding_df = pd.concat([pd.DataFrame(data[section]) for section in ('release_id','bitmap','features')],axis=1)
    image_embedding_df.columns = ['release_id', 'bitmap'] + ['feature_%s' % i for i in range(1,1281)]

In [ ]:
from cuml import UMAP

In [ ]:
scaler = StandardScaler()
umap = UMAP(n_components=10)
image_embeddings_scaled = scaler.fit_transform(image_embedding_df.loc[:,['feature_%s' % i for i in range(1,1281)]])


In [ ]:
image_embeddings_reduced = umap.fit_transform(image_embeddings_scaled)

In [ ]:
image_embeddings_reduced = pd.concat([
      image_embedding_df.loc[:,'release_id'],
      pd.DataFrame(
          image_embeddings_reduced,
          columns = ['images_umap_%s' % i for i in range(image_embeddings_reduced.shape[1])]
      )],
      axis=1
)


Combining datasets

In [ ]:
df = api_df.merge(extracted_df,how='inner',on='release_id')
df = df.merge(image_embeddings_reduced,how='inner',on='release_id')

In [ ]:
del api_df, extracted_df, image_embedding_df, image_embeddings_scaled, image_embeddings_reduced

In [ ]:
col_set = {
    'format': {
        'description': 'format_description_', 
        'name': 'format_name_', 
        'text': ('format_text_clean'),
        'quantity': ('format_quantity')
    },
    'geography': {
        'superregion': 'superregion_',
        'region': 'region_',
        'country': 'country_'
    },
    'timeperiod': {
        'period': 'period_',
        'era': 'era_'
    },
    'genre': 'genre_',
    'style': 'style_',
    'null': None,
    'indicator': lambda x: x.max() == 1 and x.min() == 0,
    'image_embedding': 'image_umap_'
}
column_store = ColumnStore()
column_store.fit(df,col_set)

## Preparing for Machine Learning

In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(df.drop('market_value',axis=1),df.market_value)

In [ ]:
del df

## Setting up Dummy Regression for baseline

In [ ]:
from sklearn.dummy import DummyRegressor

In [ ]:
dummy_val = GridSearchCV(
    DummyRegressor(),
    cv=5,
    param_grid = {
        'strategy': ['mean','median']
    },
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=5
)

In [ ]:
dummy_val.fit(X_tr,np.log(y_tr).values)

In [ ]:
dummy_val.best_score_

In [ ]:
y_mean_pred = np.full(shape=y_te.shape,fill_value=np.mean(y_te))

In [ ]:
r2_score(y_te,y_mean_pred)

In [ ]:
mean_absolute_error(y_te,y_mean_pred)

Ridge Regression

In [ ]:
full_information_transformer = ColumnTransformer(transformers=[
    ('units_for_sale_imputer', SimpleImputer(strategy='constant',fill_value=0),['units_for_sale']),
    ('average_rating_imputer', SimpleImputer(strategy='mean'),['average_rating']),
    ('year_encoder', OneHotEncoder(dtype=np.uint8), ['year'])
], remainder='passthrough')

In [ ]:
record_store_lr_transformer = ColumnTransformer(transformers=[
    ('year_encoder', OneHotEncoder(dtype=np.uint8,handle_unknown='ignore'), ['year'])
], remainder='passthrough')

In [ ]:
record_store_lr_removal_columns = [
    'market_price','units_for_sale','have','want','average_rating','rating_count','last_sold','lowest','median',
    'highest','track_titles','country','genre','style','community_have','community_want','formats','master_id','thumb_url',
    'release_url','format_description','format_text','days_since_last_sale','title','format_name'
    ]

In [ ]:
from cuml import LinearRegression

In [ ]:
record_store_lr_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist','label'])),
    ('record_store_column_remover', ColumnRemover(record_store_lr_removal_columns)),
    ('preprocessing',record_store_lr_transformer),
    ('scaler', StandardScaler()),
    ('lr', GridSearchCV(
        LinearRegression(),
        param_grid={
            'normalize':[False,True]
        },
        cv=KFold(n_splits=5,shuffle=True),
        scoring='neg_mean_absolute_error',
        verbose=5,
        n_jobs=-1
    ))
])

In [ ]:
record_store_lr_pipe.fit(X_tr,np.log(y_tr).values)

In [ ]:
record_store_lr_pipe['lr'].best_score_

In [ ]:
lr_pred = record_store_lr_pipe.predict(X_te)

In [ ]:
r2_score(y_te,np.exp(lr_pred))

In [ ]:
mean_absolute_error(y_te,np.exp(lr_pred))

In [ ]:
from cuml import Ridge
from sklearn.model_selection import KFold

In [ ]:
record_store_ridge_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist','label'])),
    ('record_store_column_remover', ColumnRemover(record_store_lr_removal_columns)),
    ('preprocessing',record_store_lr_transformer),
    ('scaler', StandardScaler()),
    ('ridge', GridSearchCV(
        Ridge(normalize=False,solver='eig'),
        param_grid={
            'alpha': np.linspace(200,300,50),
        },
        cv=KFold(n_splits=5,shuffle=True),
        scoring='neg_mean_absolute_error',
        verbose=5,
        n_jobs=-1
    ))
])

In [ ]:
record_store_ridge_pipe.fit(X_tr,np.log(y_tr).values)

In [ ]:
record_store_ridge_pipe['ridge'].best_params_

In [ ]:
ridge_pred = record_store_ridge_pipe.predict(X_te)

In [ ]:
r2_score(y_te,np.exp(ridge_pred))

In [ ]:
mean_absolute_error(y_te,np.exp(ridge_pred))

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
random_forest_param_grid = {
    'n_trees': np.linspace(50,350,50),
    'max_depth': np.linspace(10,30,5)
}

In [ ]:
record_store_random_forest_removal_columns = [
    'market_price','units_for_sale','have','want','average_rating','rating_count','last_sold','lowest','median',
    'highest','track_titles','country','genre','style','community_have','community_want','formats','master_id','thumb_url',
    'release_url','format_description','format_text','days_since_last_sale','title','format_name'
    ]

In [ ]:
record_store_random_forest_transformer = ColumnTransformer([
    ('dim_reduce_country',UMAP(n_components=10),list(column_store._country)),
    ('dim_reduce_genre', UMAP(n_components=5, list(column_store._genre))),
    ('dim_reduce_style', UMAP(n_components=10,list(column_store._style))),
    ('dim_reduce_format_description',UMAP(n_components=10, list(column_store._format_description)))
])

In [ ]:
from cuml import RandomForestRegressor

In [ ]:
random_forest_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist','label'])),
    ('record_store_column_remover', ColumnRemover(record_store_random_forest_removal_columns)),
    #('preprocessing',record_store_lr_transformer),
    ('scaler', StandardScaler()),
    ('random_forest', GridSearchCV(
        RandomForestRegressor(),
        cv=KFold(n_splits=5,shuffle=True),
        param_grid={
            'max_depth': [20]
        },
        verbose=50,
        scoring='neg_mean_absolute_error',
        n_jobs=-1,
    ))
])

In [ ]:
random_forest_pipe.fit(X_tr,np.log(y_tr).values)

## Extremely Randomized Trees

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
extra_trees_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist_clean','label_clean'])),
    ('record_store_column_remover', ColumnRemover(record_store_ridge_removal_columns)),
    ('preprocessing',record_store_transformer),
    ('scaler', StandardScaler()),
    ('random_forest', GridSearchCV(
        ExtraTreesRegressor(
            random_state=0,
            n_jobs=4,
            verbose=50
        ),
        param_grid=random_forest_param_grid,
        verbose=50
    ))
])

In [ ]:
extra_trees_pipe.fit(X_tr,np.log(y_tr))

## Catboost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
record_store_catboost_removal_columns = (
    'market_price','units_for_sale','have','want','average_rating','rating_count','last_sold','lowest','median',
    'highest','track_titles','community_have','community_want','formats','master_id','thumb_url',
    'release_url','artist','title','format_description','format_text_clean', 'no_of_days_since_last_sale'
)


In [ ]:
catboost_param_grid = {
        'depth': [4,7,10],
        'learning_rate' : np.logspace(-4,-1,5),
        'l2_leaf_reg': [1,4,9],
        'iterations': [100,300,500]
}

In [ ]:
catboost_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist_clean','label_clean'])),
    ('record_store_column_remover', ColumnRemover(record_store_catboost_removal_columns)),
    ('scaler', StandardScaler()),
    ('grid_search', GridSearchCV(
        CatBoostRegressor(
            random_state=0,
            cat_features=['year','format_text']),
        param_grid=catboost_param_grid,verbose=5)
    )
])

In [ ]:
catboost_pipe.fit(X_tr,np.log(y_tr))